In [1]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-143629
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-143629


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_name = "cpu-cluster"

# checks to see if compute target already exists in workspace, else create it
try:
    cpu_compute_target = ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                   vm_priority="dedicated", 
                                                   min_nodes=0, 
                                                   max_nodes=4)
    cpu_compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)

cpu_compute_target.wait_for_completion(show_output=True)



Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.03, 1),
    "--max_iter": choice(50, 100, 150, 200, 300)
})

# Specify a Policy (early stopping)
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", compute_target=cpu_compute_target, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, policy=policy, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20, max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

In [6]:
import joblib
# Get your best run and save the model from that run.

# RunDetails(hyperdrive_run).show()
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_cd162a63-6446-4066-9150-79b39e181b5c_2,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
metrics = best_run.get_metrics()
print(metrics)
model = best_run.register_model(model_name = 'hyperdrive_model', model_path='./outputs/model.joblib')
model

{'Regularization Strength:': 0.14533763994205545, 'Max iterations:': 150, 'Accuracy': 0.9096611026808296}


Model(workspace=Workspace.create(name='quick-starts-ws-143629', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-143629'), name=hyperdrive_model, id=hyperdrive_model:2, version=2, tags={}, properties={})

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

bank_marketing_train = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
train_data = bank_marketing_train.to_pandas_dataframe()
train_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [5]:
from train import clean_data

# Use the clean_data function to clean the data.
x, y = clean_data(bank_marketing_train)

   age          job  marital    education  default housing loan    contact  \
0   57   technician  married  high.school       no      no  yes   cellular   
1   55      unknown  married      unknown  unknown     yes   no  telephone   
2   33  blue-collar  married     basic.9y       no      no   no   cellular   
3   36       admin.  married  high.school       no      no   no  telephone   
4   27    housemaid  married  high.school       no     yes   no   cellular   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         1      failure         -1.8   
1   may         thu  ...         2    999         0  nonexistent          1.1   
2   may         fri  ...         1    999         1      failure         -1.8   
3   jun         fri  ...         4    999         0  nonexistent          1.4   
4   jul         fri  ...         2    999         0  nonexistent          1.4   

   cons.price.idx  cons.conf.idx  euribor3m 

In [6]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=bank_marketing_train,
    label_column_name='y',
    n_cross_validations=3)

In [8]:
# !pip install -r /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt

In [9]:
# Submit your automl run

automl = exp.submit(config=automl_config, show_output=True)
RunDetails(automl).show()
automl.wait_for_completion(show_output=True)

No run_configuration provided, running on local with default configuration


ConfigException: ConfigException:
	Message: Module 'azureml.train.automl.runtime' is required in the current environment for running Remote or Local (in-process) runs. Please install this dependency (e.g. `pip install azureml.train.automl.runtime`) or provide a RunConfiguration.
	InnerException: ModuleNotFoundError: No module named 'azureml.train.automl.runtime'
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Module 'azureml.train.automl.runtime' is required in the current environment for running Remote or Local (in-process) runs. Please install this dependency (e.g. `pip install azureml.train.automl.runtime`) or provide a RunConfiguration.",
        "target": "compute_target",
        "inner_error": {
            "code": "NotSupported",
            "inner_error": {
                "code": "IncompatibleOrMissingDependency"
            }
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###